In [2]:
#so we are bulding separate models
#importing basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns

#machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV,Lasso
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.ensemble import GradientBoostingRegressor
 

In [3]:
dat = pd.read_csv(r'C:\Users\Aditya\Desktop\CS data competition\Imputed.csv')

In [4]:
#filtering for sj

sj = dat[dat['city']=='iq']

In [20]:
#removing obvious variable due to high correlation with other variables, OR low correlation with target variable
var_remove = ['ndvi_ne_x','ndvi_nw_x','ndvi_se_x','ndvi_sw_x','reanalysis_air_temp_k_x','reanalysis_sat_precip_amt_mm_x','reanalysis_relative_humidity_percent_x','reanalysis_dew_point_temp_k_x']

In [21]:
sj_new = sj.drop(var_remove,axis=1)

In [22]:
sj_new.head()

,city,year,weekofyear,week_start_date,precipitation_amt_mm_x,reanalysis_avg_temp_k_x,reanalysis_max_air_temp_k_x,reanalysis_min_air_temp_k_x,reanalysis_precip_amt_kg_per_m2_x,reanalysis_specific_humidity_g_per_kg_x,reanalysis_tdtr_k_x,station_avg_temp_c_x,station_diur_temp_rng_c_x,station_max_temp_c_x,station_min_temp_c_x,station_precip_mm_x,total_cases
930,iq,2000,26,7/1/2000,25.41,298.450000,307.3,293.1,43.19,16.651429,8.928571,26.400000,10.775000,32.5,20.7,3.0,0
931,iq,2000,27,7/8/2000,60.61,298.428571,306.6,291.1,46.00,16.862857,10.314286,26.900000,11.566667,34.0,20.8,55.6,0
932,iq,2000,28,7/15/2000,55.52,297.392857,304.5,292.6,64.77,17.120000,7.385714,26.800000,11.466667,33.0,20.7,38.1,0
933,iq,2000,29,7/22/2000,5.60,296.228571,303.6,288.6,23.96,14.431429,9.114286,25.766667,10.533333,31.5,14.7,30.0,0
934,iq,2000,30,7/29/2000,62.76,297.635714,307.0,291.5,31.80,15.444286,9.500000,26.600000,11.480000,33.3,19.1,4.0,0


In [23]:
sj_new.shape

(516, 17)

In [24]:
X = sj_new.drop(['city','total_cases','week_start_date'], axis=1)
y = sj_new['total_cases']

In [25]:
# Set seed for reproducibility
SEED = 123
# Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2248,random_state=SEED)


In [65]:
X_test.shape

(116, 14)

In [27]:
#using random forest to predict by hyperparameter tuning
# Import RandomForestRegressor

# Set seed for reproducibility
SEED_rf = 1
# Instantiate a random forests regressor 'rf'
rf = RandomForestRegressor(random_state= SEED_rf)

# Inspect rf' s hyperparameters
rf.get_params()

# Basic imports

# Define a grid of hyperparameter 'params_rf'
params_rf = {
'n_estimators': [50, 100, 200,300], #number of trees in the forest
'max_depth': [4, 6, 8,10],
'min_samples_leaf': [0.05,0.1, 0.2],
'max_features': ['log2','sqrt']
}
# Instantiate 'grid_rf'
grid_rf = GridSearchCV(estimator=rf,param_grid=params_rf,cv=5,scoring='neg_mean_squared_error',verbose=1,n_jobs=-1)

# Fit 'grid_rf' to the training set
grid_rf.fit(X_train, y_train)


# Extract best hyperparameters from 'grid_rf'
best_hyperparams = grid_rf.best_params_
print('Best hyerparameters:\n', best_hyperparams)


# Extract best model from 'grid_rf'
best_model = grid_rf.best_estimator_
# Predict the test set labels
y_pred = best_model.predict(X_test)
# Evaluate the test set RMSE
rmse_test = MAE(y_test, y_pred)
# Print the test set RMSE
print('Test set MAE of rf: {:.2f}'.format(rmse_test))


Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   10.9s


Best hyerparameters:
 {'max_depth': 6, 'max_features': 'log2', 'min_samples_leaf': 0.05, 'n_estimators': 50}
Test set MAE of rf: 6.36


[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   11.9s finished


In [28]:
#parameters = {'alpha': [1e-10,1e-4,1e-1,0.5,0.8,1,5,10,20]}
lasso = Lasso(max_iter=10000, normalize=True)


lassocv = LassoCV(alphas=None, cv=10, max_iter=10000, normalize=True)
lassocv.fit(X_train, y_train)
lasso.set_params(alpha=lassocv.alpha_)
print("Alpha=", lassocv.alpha_)
lasso.fit(X_train, y_train)
print("mae = ",MAE(y_test, lasso.predict(X_test)))
print("best model coefficients:")
pd.Series(lasso.coef_, index=X.columns)

#y_pred = best_model.predict(X_test)
#Evaluate the test set RMSE
#rmse_test = MAE(y_test, y_pred)


Alpha= 0.02780519058921124
mae =  6.740960449263898
best model coefficients:


year                                       0.313637
weekofyear                                 0.000000
precipitation_amt_mm_x                     0.000000
reanalysis_avg_temp_k_x                    0.000000
reanalysis_max_air_temp_k_x               -0.000000
reanalysis_min_air_temp_k_x                0.238760
reanalysis_precip_amt_kg_per_m2_x          0.000000
reanalysis_specific_humidity_g_per_kg_x    0.534324
reanalysis_tdtr_k_x                       -0.058681
station_avg_temp_c_x                       0.000000
station_diur_temp_rng_c_x                  0.000000
station_max_temp_c_x                       0.319839
station_min_temp_c_x                       0.469142
station_precip_mm_x                        0.000000
dtype: float64

In [34]:
#Gradient Boosting

gbt = GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=SEED)
# Fit 'gbt' to the training set
gbt.fit(X_train, y_train)
# Predict the test set labels
y_pred = gbt.predict(X_test)
# Evaluate the test set RMSE
mae_test = MAE(y_test, y_pred)

print(mae_test)


5.636550586279714


In [31]:
#taking coefficients from lasso only

lasso_var = ['year','reanalysis_specific_humidity_g_per_kg_x','reanalysis_min_air_temp_k_x','reanalysis_tdtr_k_x','station_max_temp_c_x','station_min_temp_c_x']

gbt_l = GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=SEED)
# Fit 'gbt' to the training set
gbt_l.fit(X_train[lasso_var], y_train)
# Predict the test set labels
y_pred_l = gbt_l.predict(X_test[lasso_var])
# Evaluate the test set RMSE
mae_test_l = MAE(y_test, y_pred_l)

print(mae_test_l)

6.5714760889971435


In [35]:
#GB gives the best MAE

#importing the imputed test file now :))))
test_file = pd.read_csv(r'C:\Users\Aditya\Desktop\CS data competition\Imputed_test.csv')

test_file_sj = test_file[test_file['city']=='iq']


In [38]:
test_file.shape

(414, 31)

In [47]:
var_remove = ['ndvi_ne_x','ndvi_nw_x','ndvi_se','ndvi_sw','reanalysis_air_temp_k','reanalysis_sat_precip_amt_mm','reanalysis_relative_humidity_percent','reanalysis_dew_point_temp_k','city','week_start_date']

In [48]:
test_file_sj_new = test_file_sj.drop(var_remove,axis = 1)


In [57]:
selected_col = ['year', 'weekofyear', 'precipitation_amt_mm',
       'reanalysis_avg_temp_k', 'reanalysis_max_air_temp_k',
       'reanalysis_min_air_temp_k', 'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c_x', 'station_diur_temp_rng_c_x',
       'station_max_temp_c_x', 'station_min_temp_c_x', 'station_precip_mm_x']

In [58]:
test_file_sj_new_2 = test_file_sj_new[selected_col]

In [59]:
gbt_l = GradientBoostingRegressor(n_estimators=300, max_depth=3, random_state=SEED)
# Fit 'gbt' to the training set
gbt_l.fit(X, y)
# Predict the test set labels
y_pred_l = gbt_l.predict(test_file_sj_new_2)
# Evaluate the test set RMSE
#mae_test_l = MAE(y_test, y_pred_l)

#print(mae_test_l)

In [60]:
y_pred_l

array([ 3.88980962,  0.97716804,  4.78273846,  1.88110962,  2.52258042,
        2.57311831,  3.86595321,  4.16757945,  1.15561024,  1.8053184 ,
        0.66967414,  1.4948294 ,  5.96962403, -1.11057746,  2.9036038 ,
        7.2159752 , 15.16422555,  7.11089422,  9.47305555,  1.53479341,
        7.98040777,  5.86928361,  2.44079749,  7.50440085,  7.32690015,
       -5.91391936, -3.81033745, 20.8452697 , 17.39340001, 19.87674985,
       20.83046683, 19.6502246 , 17.68398552, 15.10479388, 13.21162314,
       14.86391494, 10.49919386,  7.06004038,  8.68371695,  9.28248566,
        4.2683632 ,  4.14336557,  2.39260899,  2.61590951,  3.58619614,
        6.54289545,  2.10960544,  2.56146822,  3.08455118,  6.41061979,
        3.65798042,  1.60608346,  5.11144757,  1.17550537,  2.04433426,
        2.62586042,  1.88588739,  3.04565912,  9.6816798 ,  3.49785098,
        7.9180059 ,  4.96776243,  2.45396609,  5.20863581,  5.26269163,
        1.06291905,  4.81536894,  6.34207946,  3.92689484, 11.12

In [62]:
test_file_sj['Output'] = y_pred_l

C:\Users\Aditya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
test_file_sj

,city,year,weekofyear,week_start_date,ndvi_ne_x,ndvi_nw_x,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,station_min_temp_c_x,station_precip_mm_x,ndvi_ne_y,ndvi_nw_y,station_avg_temp_c_y,station_diur_temp_rng_c_y,station_max_temp_c_y,station_min_temp_c_y,station_precip_mm_y,Output
258,iq,2010,26,2010-07-02,0.183783,0.142500,0.225129,0.150214,82.29,297.648571,...,21.5,11.2,0.230845,0.186582,27.257778,10.683889,33.350000,21.100000,18.175000,3.889810
259,iq,2010,27,2010-07-09,0.291657,0.272267,0.330700,0.320914,25.30,298.224286,...,21.2,8.9,0.233838,0.225696,26.437500,10.075000,31.466667,20.600000,14.366667,0.977168
260,iq,2010,28,2010-07-16,0.208543,0.366457,0.212629,0.255514,62.14,297.955714,...,21.6,22.6,0.243929,0.321267,26.475000,10.746429,32.400000,20.300000,19.200000,4.782738
261,iq,2010,29,2010-07-23,0.089286,0.063214,0.122057,0.081957,47.80,295.715714,...,14.2,4.8,0.199157,0.229362,26.583333,10.275000,32.900000,18.100000,14.400000,1.881110
262,iq,2010,30,2010-07-30,0.306100,0.327683,0.250086,0.267914,56.30,298.502857,...,16.9,3.0,0.302438,0.308147,26.294444,11.416667,32.666667,18.400000,16.966667,2.522580
263,iq,2010,31,2010-08-06,0.342020,0.304857,0.332357,0.345729,16.95,298.611429,...,20.0,92.2,0.328326,0.319090,27.211111,12.133333,33.733333,20.366667,32.766667,2.573118
264,iq,2010,32,2010-08-13,0.332217,0.384817,0.356600,0.353917,9.88,298.305714,...,18.5,0.0,0.314877,0.339977,26.817778,12.331111,34.200000,19.066667,0.000000,3.865953
265,iq,2010,33,2010-08-20,0.221814,0.224100,0.273086,0.231243,15.76,298.724286,...,21.0,20.3,0.232008,0.215913,27.541667,12.175000,34.466667,20.200000,10.500000,4.167579
266,iq,2010,34,2010-08-27,0.319043,0.315871,0.326700,0.344057,7.82,299.914286,...,20.0,32.9,0.338532,0.351014,27.475000,12.216667,34.700000,18.933333,32.900000,1.155610
267,iq,2010,35,2010-09-03,0.342014,0.301929,0.314700,0.381000,59.06,300.224286,...,21.0,8.5,0.285481,0.254367,27.971667,12.340000,35.233333,19.766667,16.866667,1.805318


In [64]:
test_file_sj.to_csv(r'C:\Users\Aditya\Desktop\CS data competition\Predictions_iq.csv', index = False)